In [2]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup
import time
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [96]:
matches_23_24 = pd.read_csv("teams_pl_23-24_fbref.csv")
matches_22_23 = pd.read_csv("teams_pl_22-23_fbref.csv")
matches_21_22 = pd.read_csv("teams_pl_21-22_fbref.csv")
matches_20_21 = pd.read_csv("teams_pl_20-21_fbref.csv")
matches_19_20 = pd.read_csv("teams_pl_19-20_fbref.csv")
matches_18_19 = pd.read_csv("teams_pl_18-19_fbref.csv")
matches = pd.concat([matches_23_24, matches_22_23, matches_21_22, matches_20_21, matches_19_20, matches_18_19], ignore_index=True)
stadiums = pd.read_csv("stadiums.csv")
pd.set_option('display.max_columns', None)
matches.rename(columns={"venue_date": "date", "venue_time": "time"}, inplace=True)
matches = matches.drop(columns = ["home_shirtnumber", "home_nationality", "home_position", "home_age", "away_shirtnumber", "away_nationality", "away_position", "away_age"])
matches = matches.drop(columns=["home_captain", "away_captain", "home_manager", "away_manager"])

In [97]:
matches["minute"] = 90
matches.head(2)
columns_needed = ["season", "date", "time", "round", "attendance_value", "referee", "formation_home", "formation_away", "home_team", "away_team"]

In [98]:
matches = matches[[col for col in matches.columns if ('goals' in col or 'shots' in col or 'passes' in col or col in columns_needed) and 'pct' not in col]]

In [99]:
matches.head(2)

,season,date,time,round,attendance_value,referee,formation_home,formation_away,home_team,away_team,home_goals,home_shots,home_shots_on_target,home_passes_completed,home_passes,home_progressive_passes,home_passes_total_distance,home_passes_progressive_distance,home_passes_completed_short,home_passes_short,home_passes_completed_medium,home_passes_medium,home_passes_completed_long,home_passes_long,home_assisted_shots,home_passes_into_final_third,home_passes_into_penalty_area,home_passes_live,home_passes_dead,home_passes_free_kicks,home_passes_switches,home_passes_offsides,home_passes_blocked,home_blocked_shots,home_blocked_passes,home_passes_received,home_progressive_passes_received,home_own_goals,away_goals,away_shots,away_shots_on_target,away_passes_completed,away_passes,away_progressive_passes,away_passes_total_distance,away_passes_progressive_distance,away_passes_completed_short,away_passes_short,away_passes_completed_medium,away_passes_medium,away_passes_completed_long,away_passes_long,away_assisted_shots,away_passes_into_final_third,away_passes_into_penalty_area,away_passes_live,away_passes_dead,away_passes_free_kicks,away_passes_switches,away_passes_offsides,away_passes_blocked,away_blocked_shots,away_blocked_passes,away_passes_received,away_progressive_passes_received,away_own_goals
0,2023-2024,2023-08-11,20:00,1,21572.0,Craig Pawson,5-4-1,4-2-3-1,Burnley,Manchester City,0,6,1,313,395,18,5807,1910,133,148,134,151,40,74,5,14,3,356,39,9,4,0,8,5,7,313,18,0,3,17,8,655,740,28,10790,3194,325,339,252,277,51,84,14,42,10,703,36,10,1,1,7,2,7,632,28,0
1,2023-2024,2023-08-12,12:30,1,59984.0,Michael Oliver,4-3-3,3-4-3,Arsenal,Nottingham Forest,2,15,7,716,807,58,11400,3204,367,393,306,334,35,55,13,61,13,756,49,13,1,2,7,1,5,712,58,0,1,6,2,154,228,14,3239,1786,57,76,68,83,25,53,5,15,5,193,34,14,0,1,5,5,5,151,14,0


In [112]:
def simulate(df, total_minutes=90):
    expanded_data = []
    for _, row in df.iterrows():      
        # Set up initial values
        home_goals = 0
        away_goals = 0
        home_shots = 0
        away_shots = 0
        cumsum = {}
        for col in df.columns:
            cumsum[col] = 0
        for minute in range(1, total_minutes + 1):
            simulated_row = row.to_dict()
            simulated_row['minute'] = minute
            
                

            for col in df.columns:
                if col not in columns_needed:
                    cumsum[col] += np.random.poisson(row[col] / total_minutes)

                    simulated_row[col] = cumsum[col]
                else:
                    simulated_row[col] = row[col]

            expanded_data.append(simulated_row)
    
    return pd.DataFrame(expanded_data)

In [115]:
simulated_matches = simulate(matches)
# pd.set_option('display.max_rows', 20)

In [117]:
simulated_matches.to_csv("simulated_matches.csv", index=False)

In [119]:
n = len(simulated_matches)
split_size = n // 3

# Split the DataFrame
df1 = simulated_matches.iloc[:split_size]
df2 = simulated_matches.iloc[split_size:2 * split_size]
df3 = simulated_matches.iloc[2 * split_size:]

# Save smaller DataFrames as CSV
df1.to_csv("simulated_matches1.csv", index=False)
df2.to_csv("simulated_matches2.csv", index=False)
df3.to_csv("simulated_matches3.csv", index=False)